# Agenda: Strings in Pandas

1. How does Pandas store strings?
2. How to work with strings using `.str` (the "string accessor")
3. A lot of string methods that we might want to use
4. String methods on non-strings
5. Regular expressions and the string methods
6. Memory usage
7. Extension types
8. PyArrow

# Pandas and strings

We know that normally, Pandas stores its data in NumPy.

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
s = Series([10, 20, 30])
s

0    10
1    20
2    30
dtype: int64

In [4]:
s.values   # get the NumPy array back from behind the Pandas series

array([10, 20, 30])

In [5]:
s.values.dtype

dtype('int64')

In [6]:
s = Series([10.5, 20.5, 30.5])
s

0    10.5
1    20.5
2    30.5
dtype: float64

In [7]:
s.values.dtype

dtype('float64')

In [8]:
# let's say that I want to create a NumPy array of strings. Can I?

a = np.array('hello out there'.split())
a

array(['hello', 'out', 'there'], dtype='<U5')

When NumPy stores strings, it does so in C-type memory. An array (including a NumPy array), by definition, has elements that are 
all of the same type. In this case, all of the elements of `a` are of type `<U5`, which means: Up to 5 Unicode characters.

In [9]:
a[0] = 'goodbye'

In [10]:
a

array(['goodb', 'out', 'there'], dtype='<U5')

For this reason, and many others, we don't want to use NumPy's strings. They are fragile, in that we can accidentally
remove characters, and they don't have the flexiblity that we know and love from Python strings.

The Pandas solution to this problem is: Use Python strings!

In [11]:
s = Series('hello out there'.split())
s

0    hello
1      out
2    there
dtype: object

A dtype of `object` means: NumPy has a pointer ("reference") to a Python object. Normally, and most of the time, 
if you see a dtype of `object`, it's because we're working with strings.

The good news is then that we can retrieve these strings and run Python methods on them. Also good news is that
the size of the string isn't constrained by the memory allocated by NumPy.



In [12]:
s.loc[0] = 'goodbye'

In [13]:
s

0    goodbye
1        out
2      there
dtype: object

In [15]:
type(s.loc[0])

str

Now that we know that we have Python strings... how can we invoke methods on them? Should we just use a `for` loop
and iterate over each string, doing something to it?

In [16]:
for one_word in s:
    print(one_word.upper())

GOODBYE
OUT
THERE


# Don't do this (for several reasons)

1. Using a `for` loop on a series is almost always a bad idea. Each iteration retrieves the value from NumPy and creates a Python object based on it.
2. We lose the index, which might be important in this particular series.

So how should we work with our strings?

Answer: Via the `.str` accessor. Meaning: If we apply `.str` to our series, we get back an object on which we can run
methods. It will then effectively run the method in a vectorized fashion, faster (we can hope) than would be the case
using a `for` loop.

In [17]:
s.str

In [18]:
s.str.len()  # this will invoke len() on each string, giving us back a new series

0    7
1    3
2    5
dtype: int64

In [19]:
s = Series('hello out there'.split(),
          index=list('abc'))
s

a    hello
b      out
c    there
dtype: object

In [20]:
s.str.len()  # this will invoke len() on each string, giving us back a new series WITH THE original's index!

a    5
b    3
c    5
dtype: int64

# Exercise: Longer-than-average words

1. Create a series of 10 strings (words), each of a different length.
2. Calculate the mean length of a word in the series.
3. Find all of the words that are longer than the mean.

In [21]:
s = Series('this is a fantastic example of strings in action everywhere'.split())
s

0          this
1            is
2             a
3     fantastic
4       example
5            of
6       strings
7            in
8        action
9    everywhere
dtype: object

In [22]:
s.str.len()

0     4
1     2
2     1
3     9
4     7
5     2
6     7
7     2
8     6
9    10
dtype: int64

In [23]:
s.str.len().mean()

5.0

In [24]:
s.str.len().describe()

count    10.000000
mean      5.000000
std       3.231787
min       1.000000
25%       2.000000
50%       5.000000
75%       7.000000
max      10.000000
dtype: float64

In [26]:
# which words are longer than 5 characters?
s.str.len() > 5

0    False
1    False
2    False
3     True
4     True
5    False
6     True
7    False
8     True
9     True
dtype: bool

In [31]:
# which words are longer than the mean of s's words' lengths?
s.loc[
    s.str.len() > s.str.len().mean()
]

3     fantastic
4       example
6       strings
8        action
9    everywhere
dtype: object

In [34]:
# what if our strings contain leading/trailing whitespace?

s = Series(['    abcd   ', 'efgh    ', '    ijkl    ', ' abcd ', ' ijkl      ', '       ijkl  ', '     abcd '])
s

0          abcd   
1         efgh    
2         ijkl    
3            abcd 
4       ijkl      
5           ijkl  
6            abcd 
dtype: object

In [35]:
s.str.len()

0    11
1     8
2    12
3     6
4    11
5    13
6    10
dtype: int64

In [36]:
s.value_counts()   # how often does each string appear in our series?

    abcd         1
efgh             1
    ijkl         1
 abcd            1
 ijkl            1
       ijkl      1
     abcd        1
Name: count, dtype: int64

In [42]:
# I can run another string method, s.str.strip()

s.str.strip().value_counts()

abcd    3
ijkl    3
efgh    1
Name: count, dtype: int64

In [43]:
s

0          abcd   
1         efgh    
2         ijkl    
3            abcd 
4       ijkl      
5           ijkl  
6            abcd 
dtype: object

# Some useful methods for cleaning our data:

- `.str.strip()` -- no argument means "remove all whitespace from the front and back of the string," but we can pass a string to it, in which case all of its characters are stripped from the front and back
- `.str.lower()` -- return a lowercase version
- `.str.upper()` -- return an uppercase version

In [44]:
s = Series(['    abcd   ', 'efgh    ', '    iJkL    ', ' ABCD ', ' Ijkl      ', '       ijkL  ', '     abcd '])


In [46]:
s.str.strip().value_counts()

abcd    2
efgh    1
iJkL    1
ABCD    1
Ijkl    1
ijkL    1
Name: count, dtype: int64

In [48]:
(
    s
    .str.strip()     # remove leading/trailing whitespace
    .str.lower()     # get back a new series of strings, all lowercase, based on output from .str.strip()
    .value_counts()  # runs on the result from str.lower
)

abcd    3
ijkl    3
efgh    1
Name: count, dtype: int64

# Exercise: Alice words

1. Create a series based on the *words* in Alice in Wonderland. (You can use `read` on this file, without worrying about the length.)
2. Find the 10 most common words in the book, without any transformation.
3. Find the 10 most common words in the book, removing punctuation (`string.punctuation`) from the front and back of words.
4. Find the 10 most common words in the book, lowercasing the words *and* removing punctuation.

In [49]:
s = Series(open('alice-in-wonderland.txt').read().split())

In [50]:
len(s)

12763

In [51]:
s.loc[:15]

0            ﻿The
1         Project
2       Gutenberg
3           EBook
4              of
5           Alice
6              in
7     Wonderland,
8              by
9           Lewis
10        Carroll
11           This
12          eBook
13             is
14            for
15            the
dtype: object

In [52]:
s.value_counts()

the            732
and            362
a              321
to             311
of             300
              ... 
present--at      1
morning,         1
then."           1
sternly.         1
newsletter       1
Name: count, Length: 3408, dtype: int64

In [54]:
# 3. Find the 10 most common words in the book, removing punctuation (`string.punctuation`) from the front and back of words.

import string

(
    s
    .str.strip(string.punctuation)
    .value_counts()
    .head(10)
)


the      735
and      384
a        322
to       320
of       303
in       214
she      201
was      167
Alice    166
it       164
Name: count, dtype: int64

In [55]:
# 4. Find the 10 most common words in the book, lowercasing the words *and* removing punctuation.

import string

(
    s
    .str.strip(string.punctuation)
    .str.lower()
    .value_counts()
    .head(10)
)

the    807
and    404
a      328
to     327
of     318
she    237
in     227
it     183
you    171
was    168
Name: count, dtype: int64

In [57]:
# now let's ignore any word whose length is < 5.

import string

(
    s
    .str.strip(string.punctuation)
    .str.lower()
    .loc[s.str.len() >= 5]
    .value_counts()
    .head(10)
)

alice           168
project          87
little           59
gutenberg-tm     56
about            41
other            34
herself          33
there            32
works            32
rabbit           31
Name: count, dtype: int64

In [58]:
# again, but with lambda!

import string

(
    s
    .str.strip(string.punctuation)
    .str.lower()
    .loc[lambda s_ : s_.str.len() >= 5]
    .value_counts()
    .head(10)
)

alice           168
project          87
little           59
gutenberg-tm     56
about            41
other            34
herself          33
works            32
there            32
rabbit           31
Name: count, dtype: int64

In [61]:
s.str.strip(string.punctuation).value_counts().head(15)

the      735
and      384
a        322
to       320
of       303
in       214
she      201
was      167
Alice    166
it       164
said     143
you      141
I        127
with     112
her      107
Name: count, dtype: int64

In [67]:
ignore_words = 'and the if an a i to of in she was it you with her or as at all on be is'.split()

import string

(
    s
    .str.strip(string.punctuation)
    .str.lower()
    # .loc[lambda s_ : s_.str.len() >= 5]
    .loc[lambda s_: ~ s_.isin(ignore_words)]
    .value_counts()
    .head(10)
)

alice           168
said            144
that            111
this             98
project          87
for              74
had              65
not              62
little           59
gutenberg-tm     56
Name: count, dtype: int64

# Turning strings into integers

Normally, we can use `astype(int)` to get a new series back from a series of strings. But what if one or more of those
strings aren't valid ints?

In [68]:
s = Series('10 20 30'.split())
s

0    10
1    20
2    30
dtype: object

In [69]:
s.astype(int)

0    10
1    20
2    30
dtype: int64

In [70]:
# let's try something that doesn't work...

s = Series('10 20 hello 30'.split())
s

0       10
1       20
2    hello
3       30
dtype: object

In [71]:
s.astype(int)

ValueError: invalid literal for int() with base 10: 'hello'

In [73]:
# let's remove any value from the series that cannot be turned into an int
# an easy way to do that is with the str.isdigit method

(
    s
    .loc[s.str.isdigit()]
    .astype(int)
)

0    10
1    20
3    30
dtype: int64

In [74]:
# using a lambda expression

(
    s
    .loc[lambda s_: s_.str.isdigit()]
    .astype(int)
)

0    10
1    20
3    30
dtype: int64

# Examining the text

Normally, when we're using Python, we can use a few operators and methods to explore our strings. These are all available in Pandas, too, but sometimes with different names:

- `index` and `rindex`
- `startswith` and `endswith`
- `contains`, a method version of the `in` operator for regular strings

In [76]:
s = Series('hello to everyone out there in television land'.split())
s

0         hello
1            to
2      everyone
3           out
4         there
5            in
6    television
7          land
dtype: object

In [79]:
s.loc[s.str.contains('e')]

0         hello
2      everyone
4         there
6    television
dtype: object

In [80]:
# what if I want to find words that contain a or e?

s.loc[s.str.contains('e') | s.str.contains('a')]

0         hello
2      everyone
4         there
6    television
7          land
dtype: object

In [83]:
# but ... there's a better way
# here, we used a regular expression (aka "regex" or "regexp")

# in our regexp, we're saying:
# look for any one of the characters in []
# (known as a "character class") 
# if either a or e is in the string, the regexp returns True

s.loc[s.str.contains('[ae]', regex=True)]

0         hello
2      everyone
4         there
6    television
7          land
dtype: object

In [85]:
s = r'a\wb'

In [86]:
words = 'hello 10 whatever 20 this is weird !@#$!@$!asdfa whatever again'.split()

s = Series(words)
s



0            hello
1               10
2         whatever
3               20
4             this
5               is
6            weird
7    !@#$!@$!asdfa
8         whatever
9            again
dtype: object

In [90]:
# let's find all of the words that contain only letters

(
    s
    .loc[lambda s_: s_.str.contains(r'^[a-z]+$')]   # from the start, to the end, only alphanumeric characters
)

0       hello
2    whatever
4        this
5          is
6       weird
8    whatever
9       again
dtype: object

In [91]:
(
    s
    .loc[lambda s_: s_.str.contains(r'^[^a-z]+$')]   # from the start, to the end, only alphanumeric characters
)

1    10
3    20
dtype: object

# Exercise: Weird words

1. Create a series with the words from Alice in Wonderland.
2. Find words that end with a question mark. (Remember, you can use `str.endswith`)
3. Find words that contain 'ai' in them.

In [93]:
s = (
    Series(open('alice-in-wonderland.txt').read().split())
    .str.strip()
)

s.head(15)

0            ﻿The
1         Project
2       Gutenberg
3           EBook
4              of
5           Alice
6              in
7     Wonderland,
8              by
9           Lewis
10        Carroll
11           This
12          eBook
13             is
14            for
dtype: object

In [95]:
# 2. Find words that end with a question mark. (Remember, you can use `str.endswith`)

s.loc[s.str.endswith('?')]

549         end?
1435        way?
1537      dears?
1814    morning?
2129      mouse?
2166       pool?
2435       dogs?
2767      ready?
3838       here?
4534     ladder?
5384     _what_?
5610         to?
8715      TARTS?
dtype: object

In [99]:
# 3. Find words that contain 'ai' in them.

(
    s
    .str.strip(string.punctuation)
    .loc[lambda s_: s_.str.contains('ai')]
    .value_counts()
)

said                            143
again                            28
domain                            7
against                           6
paid                              6
afraid                            6
air                               5
waiting                           4
pair                              4
certain                           4
certainly                         4
obtain                            3
tail                              3
waited                            3
painting                          2
Plain                             2
disclaimer                        2
contain                           2
obtaining                         2
email                             2
remained                          2
waistcoat-pocket                  2
available                         2
explain                           2
fountains                         2
containing                        1
Domain                            1
maintaining                 

# What about using `[]`?

In [100]:
s.str[0]  # [] are turned into a method call, into __getitem__ behind the scenes

0        ﻿
1        P
2        G
3        E
4        o
        ..
12758    t
12759    h
12760    a
12761    n
12762    e
Length: 12763, dtype: object

In [101]:
# if you want the "official" method for doing that, you can use .str.get

s.str.get(0)

0        ﻿
1        P
2        G
3        E
4        o
        ..
12758    t
12759    h
12760    a
12761    n
12762    e
Length: 12763, dtype: object

In [102]:
s.str.get(-1)

0        e
1        t
2        g
3        k
4        f
        ..
12758    o
12759    r
12760    t
12761    w
12762    .
Length: 12763, dtype: object

In [103]:
# when we use [] in Python, we can pass a single number (an index) or a slice. Can I do that here?

s.str[2:5]

0         he
1        oje
2        ten
3        ook
4           
        ... 
12758       
12759     ar
12760    out
12761      w
12762    ook
Length: 12763, dtype: object

In [104]:
# if you want a more method-ish version, you can call .str.slice

s.str.slice(2, 5)

0         he
1        oje
2        ten
3        ook
4           
        ... 
12758       
12759     ar
12760    out
12761      w
12762    ook
Length: 12763, dtype: object

In [105]:
s = Series(['hello out there', 'I am still here', 'I am trapped in a series'])
s

0             hello out there
1             I am still here
2    I am trapped in a series
dtype: object

In [107]:
# How can I grab the final word from each of these series?

# how would I do it if I had one in regular Python?

s = 'I am trapped in a series'

# how can I get the final word?
s.split()[-1]

'series'

In [110]:
# can I do that with this series?
# yes, because we have str.split() inside of Pandas, too!

s = Series(['hello out there', 'I am still here', 'I am trapped in a series'])
s.str.split().str[-1]

0     there
1      here
2    series
dtype: object

# Memory usage

The whole point of using NumPy as a backend for Pandas is that it's compact. When we use Python strings, we get a lot of convenience, but we lose that compactness, because Python strings are much larger.

In [111]:
np.random.seed(0)
df = DataFrame(np.random.randint(0, 1000, [4, 3]))
df

,0,1,2
0,684,559,629
1,192,835,763
2,707,359,9
3,723,277,754


In [112]:
# how much memory is this data frame using?

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       4 non-null      int64
 1   1       4 non-null      int64
 2   2       4 non-null      int64
dtypes: int64(3)
memory usage: 228.0 bytes


In [113]:
# what if I add an index to the data frame?

df.index=list('abcd')
df

,0,1,2
a,684,559,629
b,192,835,763
c,707,359,9
d,723,277,754


In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       4 non-null      int64
 1   1       4 non-null      int64
 2   2       4 non-null      int64
dtypes: int64(3)
memory usage: 128.0+ bytes


When we ask for memory usage stats in `df.info()` or in `df.memory_usage`, we're only getting back results for the NumPy-allocated memory usage. Anything in Python's memory is ignored, because it'll take too long to calculate.

When Pandas sees that it has data (or even just part of the index!) in Python-world, it doesn't calculate it but does put a `+` next to the memory stat, so we know it.

We can force the checking of all memory, including Python objects, by passing `memory_usage='deep'`.

In [116]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       4 non-null      int64
 1   1       4 non-null      int64
 2   2       4 non-null      int64
dtypes: int64(3)
memory usage: 296.0 bytes


In [120]:
df['s'] = ['a', 'b', 'c', 'd']

In [121]:
df

,0,1,2,s
a,abcdefghij,559,629,a
b,192,835,763,b
c,707,359,9,c
d,723,277,754,d


In [122]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       4 non-null      object
 1   1       4 non-null      int64 
 2   2       4 non-null      int64 
 3   s       4 non-null      object
dtypes: int64(2), object(2)
memory usage: 803.0 bytes


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       4 non-null      object
 1   1       4 non-null      int64 
 2   2       4 non-null      int64 
 3   s       4 non-null      object
dtypes: int64(2), object(2)
memory usage: 332.0+ bytes


# `NaN` is a problem

`NaN` (not a number) is a float. If we have `NaN` in our string column, it'll work, because `object` can be any object, including a Python string or `NaN`. But it feels weird to have a float in our strings.

Pandas has defined a number of "extension classes," and these are "nullable," meaning that you can have string values + `NA` (the modern equivalent to `NaN`), and you aren't forced to change the dtype, or to use `object`. You can do this for strings by specifying the `dtype` to be `StringDType()`. Notice that it's a function!



In [124]:
s = Series('hello out there, everyone!'.split())
s

0        hello
1          out
2       there,
3    everyone!
dtype: object

In [126]:
s = Series('hello out there, everyone!'.split(), dtype=pd.StringDtype())
s

0        hello
1          out
2       there,
3    everyone!
dtype: string

In [127]:
s.loc[2] = pd.NA

In [128]:
s

0        hello
1          out
2         <NA>
3    everyone!
dtype: string

In [129]:
# what about PyArrow?
# PyArrow is a different backend, ditching NumPy in favor of storage that's more appropriate for Pandas analysis.
# we can ask for a data frame to be defined using PyArrow, rather than NumPy.

df = pd.read_csv('taxi.csv')
df.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
pickup_longitude         float64
pickup_latitude          float64
RateCodeID                 int64
store_and_fwd_flag        object
dropoff_longitude        float64
dropoff_latitude         float64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtype: object

In [130]:
filename = '/Users/reuven/Courses/Current/data/olympic_athlete_events.csv'

df = pd.read_csv(filename)

In [131]:
df.dtypes

ID          int64
Name       object
Sex        object
Age       float64
Height    float64
Weight    float64
Team       object
NOC        object
Games      object
Year        int64
Season     object
City       object
Sport      object
Event      object
Medal      object
dtype: object

In [132]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 158.9 MB


In [133]:
# let's use PyArrow as our backend!

filename = '/Users/reuven/Courses/Current/data/olympic_athlete_events.csv'

df = pd.read_csv(filename, dtype_backend='pyarrow')

In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype          
---  ------  --------------   -----          
 0   ID      271116 non-null  int64[pyarrow] 
 1   Name    271116 non-null  string[pyarrow]
 2   Sex     271116 non-null  string[pyarrow]
 3   Age     261642 non-null  int64[pyarrow] 
 4   Height  210945 non-null  int64[pyarrow] 
 5   Weight  208241 non-null  double[pyarrow]
 6   Team    271116 non-null  string[pyarrow]
 7   NOC     271116 non-null  string[pyarrow]
 8   Games   271116 non-null  string[pyarrow]
 9   Year    271116 non-null  int64[pyarrow] 
 10  Season  271116 non-null  string[pyarrow]
 11  City    271116 non-null  string[pyarrow]
 12  Sport   271116 non-null  string[pyarrow]
 13  Event   271116 non-null  string[pyarrow]
 14  Medal   39783 non-null   string[pyarrow]
dtypes: double[pyarrow](1), int64[pyarrow](4), string[pyarrow](10)
memory usage: 46.4 MB


In [135]:
df['Team']

0                  China
1                  China
2                Denmark
3         Denmark/Sweden
4            Netherlands
               ...      
271111          Poland-1
271112            Poland
271113            Poland
271114            Poland
271115            Poland
Name: Team, Length: 271116, dtype: string[pyarrow]